In [1]:
from flask import Flask, render_template, redirect
from flask import request, flash, send_from_directory, request, session, abort
from process import *
from config import FOLDER_NAME
import time, os

In [27]:
username = 'user1'
password = 'P@ssw0rd'

data = pd.read_csv(r'user.csv')   
df = pd.DataFrame(data, columns=['user', 'password'])

for i in df.index:
    user, pas = df.iloc[i]
    if user == username and pas == password:
        print('true')
    else:
        print('false')

false
false


In [6]:
sql_interval = '1 DAY'
_DB_NAME_ = "db_bat_pct2"
startDate, endDate = (pd.to_datetime('now') - pd.to_timedelta(sql_interval), pd.to_datetime('now'))

where_state = f"WHERE ts > (SELECT ts FROM {_DB_NAME_}.tb_combustion_model_generation GROUP BY ts ORDER BY ts DESC LIMIT 4, 1)"

q = f"""SELECT ts AS timestamp, tag_name AS 'desc', value AS targetValue, bias_value AS setValue, value-bias_value AS currentValue FROM {_DB_NAME_}.tb_combustion_model_generation
        {where_state}
        ORDER BY ts DESC, tag_name ASC"""

df = pd.read_sql(q, engine).replace(np.nan, 0)

for c in df.columns[-3:]:
    df[c] = np.round(df[c], 3)
df_dict = df.astype(str).to_dict('records')

last_recommendation = str(df['timestamp'].max())
df_dict

[{'timestamp': '2022-04-19 22:40:00',
  'desc': 'Burner Tilt Position 1L',
  'targetValue': '4.228',
  'setValue': '-52.975',
  'currentValue': '57.203'},
 {'timestamp': '2022-04-19 22:40:00',
  'desc': 'Burner Tilt Position 1R',
  'targetValue': '0.264',
  'setValue': '-57.658',
  'currentValue': '57.922'},
 {'timestamp': '2022-04-19 22:40:00',
  'desc': 'Burner Tilt Position 2L',
  'targetValue': '3.132',
  'setValue': '-55.868',
  'currentValue': '59.0'},
 {'timestamp': '2022-04-19 22:40:00',
  'desc': 'Burner Tilt Position 2R',
  'targetValue': '0.285',
  'setValue': '-58.715',
  'currentValue': '59.0'},
 {'timestamp': '2022-04-19 22:40:00',
  'desc': 'Burner Tilt Position 3L',
  'targetValue': '1.284',
  'setValue': '-57.716',
  'currentValue': '59.0'},
 {'timestamp': '2022-04-19 22:40:00',
  'desc': 'Burner Tilt Position 3R',
  'targetValue': '2.449',
  'setValue': '-55.7',
  'currentValue': '58.148'},
 {'timestamp': '2022-04-19 22:40:00',
  'desc': 'Burner Tilt Position 4L',
  '

In [36]:
# bg_safeguard_update()
# bg_update_notification()

q = f"""SELECT raw.f_address_no, CAST(raw.f_value AS float) as f_value, '' AS f_message, raw.f_updated_at FROM tb_tags_read_conf conf 
    LEFT JOIN tb_bat_raw raw
    ON conf.f_tag_name = raw.f_address_no 
    WHERE conf.f_description = "{config.DESC_ENABLE_COPT}" """
df_status_now = pd.read_sql(q, engine)
tag_name, status_now, message, timestamp_now = df_status_now.iloc[0].values
status_now = int(status_now)
timestamp_now = pd.to_datetime(timestamp_now).strftime('%Y-%m-%d %X')

# Get latest update on message
q = f"""SELECT notif.f_value from tb_tags_read_conf conf
    LEFT JOIN tb_bat_notif notif 
    ON conf.f_tag_name = notif.f_address_no 
    WHERE conf.f_description = "{config.DESC_ENABLE_COPT}"
    ORDER BY notif.f_updated_at DESC LIMIT 1"""
df_status_last = pd.read_sql(q, engine)
status_last = int(0) if df_status_last.notna else int(df_status_last['f_value'].values[0])
status_last

IndexError: index 0 is out of bounds for axis 0 with size 0